# Combination of Price and Position algorithm

In [27]:
import math
import pandas as pd
import numpy as np
import csv
import pandasql as ps

In [28]:
train_data = pd.read_csv("~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/train.csv")

In [29]:
test_data = pd.read_csv("~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/test_new.csv")

In [30]:
train = train_data
train_exercise = train[:100000]
train_exercise.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
7,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
8,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
9,00RL8Z82B2Z1,aff3928535f48,1541037542,10,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


In [31]:
train_exercise = train_exercise.loc[train_exercise['action_type'] == 'clickout item']

In [32]:
train_exercise.shape

(9280, 12)

In [33]:
def string_to_array(s):
    """Convert pipe separated string to array."""

    if isinstance(s, str):
        out = s.split("|")
    elif math.isnan(s):
        out = []
    else:
        raise ValueError("Value must be either string of nan")
    return out

def explode_item_price(df_in, col_explode1, col_explode2):
    """Explode column col_expl of array type into multiple rows."""

    df = df_in.copy()
    df.loc[:, col_explode1] = df[col_explode1].apply(string_to_array)
    df.loc[:, col_explode2] = df[col_explode2].apply(string_to_array)    

    df_out = pd.DataFrame(
        {col: np.repeat(df[col].values,
                        df[col_explode1].str.len())
         for col in df.columns.drop([col_explode1,col_explode2])}
    )

    df_out.loc[:, col_explode1] = np.concatenate(df[col_explode1].values)
    df_out.loc[:, col_explode2] = np.concatenate(df[col_explode2].values)  
    
    df_out.loc[:, col_explode1] = df_out[col_explode1].apply(int)
    df_out.loc[:, col_explode2] = df_out[col_explode2].apply(int)

    return df_out

In [43]:
train_exercise_expl = explode_item_price(train_exercise, 'impressions', 'prices')
train_exercise_expl.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638,95
1,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,1253714,66
2,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3367857,501
3,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,5100540,112
4,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,1088584,95


In [53]:
train_exercise_expl["POS"] = (
    train_exercise_expl
    .groupby(['user_id','session_id','timestamp','step'])
    .cumcount()
)

In [72]:
train_exercise_expl['POS_price'] = (
    train_exercise_expl
    .groupby(['user_id','session_id','timestamp','step'])
    .cumcount()
)

In [63]:
train_exercise_expl = (
    train_exercise_expl
    .groupby(['user_id','session_id','timestamp','step'])
    .apply(lambda x: x.sort_values(['prices','POS']))
    .reset_index(drop=True)
)

In [64]:
train_exercise_expl = train_exercise_expl.astype(str)

In [73]:
train_exercise_expl['is_clicked'] = np.where(train_exercise_expl['reference'] == train_exercise_expl['impressions'], 1, 0)


In [74]:
train_exercise_expl["POS"] = pd.to_numeric(train_exercise_expl["POS"])
train_exercise_expl["POS_price"] = pd.to_numeric(train_exercise_expl["POS_price"])
train_exercise_expl["is_clicked"] = pd.to_numeric(train_exercise_expl["is_clicked"])
train_exercise_expl["prices"] = pd.to_numeric(train_exercise_expl["prices"])

In [75]:
train_exercise_expl.groupby(['POS_price','is_clicked']).count()

user_id  session_id  timestamp  step  action_type  \
POS_price is_clicked                                                      
0         0              8251        8251       8251  8251         8251   
          1              1029        1029       1029  1029         1029   
1         0              8523        8523       8523  8523         8523   
          1               736         736        736   736          736   
2         0              8597        8597       8597  8597         8597   
          1               631         631        631   631          631   
3         0              8672        8672       8672  8672         8672   
          1               527         527        527   527          527   
4         0              8706        8706       8706  8706         8706   
          1               449         449        449   449          449   
5         0              8677        8677       8677  8677         8677   
          1               436         436        436   436          436   
6         0              8697        8697       8697  8697         8697   
          1               372         372        372   372          372   
7         0              8625        8625       8625  8625         8625   
          1               418         418        418   418          418   
8         0              8605        8605       8605  8605         8605   
          1               389         389        389   389          389   
9         0              8599        8599       8599  8599         8599   
          1               344         344        344   344          344   
10        0              8534        8534       8534  8534         8534   
          1               332         332        332   332          332   
11        0              8442        8442       8442  8442         8442   
          1               320         320        320   320          320   
12        0              8345        8345       8345  8345         8345   
          1               289         289        289   289          289   
13        0              8203        8203       8203  8203         8203   
          1               326         326        326   326          326   
14        0              8107        8107       8107  8107         8107   
          1               317         317        317   317          317   
15        0              8048        8048       8048  8048         8048   
          1               294         294        294   294          294   
16        0              8013        8013       8013  8013         8013   
          1               233         233        233   233          233   
17        0              7907        7907       7907  7907         7907   
          1               245         245        245   245          245   
18        0              7805        7805       7805  7805         7805   
          1               240         240        240   240          240   
19        0              7721        7721       7721  7721         7721   
          1               228         228        228   228          228   
20        0              7582        7582       7582  7582         7582   
          1               265         265        265   265          265   
21        0              7537        7537       7537  7537         7537   
          1               242         242        242   242          242   
22        0              7493        7493       7493  7493         7493   
          1               205         205        205   205          205   
23        0              7427        7427       7427  7427         7427   
          1               198         198        198   198          198   
24        0              7040        7040       7040  7040         7040   
          1               212         212        212   212          212   

                      reference  platform  city  device  current_filters  \
POS_price is_clicked                         

## Check of price by position

By every clicked item position I'm checking the average price position is it been displayed and also the ratio of the actual price compared to the average price displayed

In [76]:
GR_COLS = ["user_id", "session_id", "timestamp", "step"]

In [77]:
analysis = (
    train_exercise_expl
    .groupby(['POS','is_clicked'])
    .mean()
)

In [78]:
analysis

prices  POS_price
POS is_clicked                       
0   0           137.288111  11.401516
    1           118.944174  10.011035
1   0           126.659752  11.365457
    1           103.715909   8.612603
2   0           130.620019  11.370722
    1            95.520115   8.260057
3   0           130.205288  11.547165
    1            98.529740   8.202602
4   0           127.353320  11.233283
    1            97.804721   8.113734
5   0           126.215172  11.343218
    1            91.523002   8.021792
6   0           127.307093  11.132806
    1            86.333333   8.494152
7   0           121.097274  11.192496
    1           106.036496   8.405109
8   0           124.510692  11.420469
    1            98.967871   8.879518
9   0           123.614739  11.350885
    1           118.187755   8.873469
10  0           122.257968  11.470092
    1            97.951456   8.587379
11  0           122.565797  11.585916
    1            90.014085   7.638498
12  0           119.774488  11.424849
    1            90.827225   8.340314
13  0           116.924815  11.389553
    1           110.159509   8.607362
14  0           120.281439  11.704079
    1            95.384058   8.644928
15  0           114.845674  11.461388
    1            92.905063   8.949367
16  0           116.912040  11.834444
    1           109.921429   8.464286
17  0           117.881936  11.753546
    1            96.508772   8.236842
18  0           116.237476  11.840126
    1           130.250000   9.016667
19  0           118.670144  12.034092
    1           104.409091   8.988636
20  0           118.306679  12.042501
    1           106.415094   8.943396
21  0           121.203101  12.142783
    1           106.932039  10.466019
22  0           116.817238  12.224412
    1            96.620690   7.586207
23  0           119.798036  12.352176
    1           108.123596   9.831461
24  0           115.461259  12.265950
    1           131.089888   9.067416

In [79]:
train_exercise_expl['price_mean'] = (
    train_exercise_expl
    .groupby(GR_COLS)['prices']
    .transform('mean')
)

In [80]:
train_exercise_expl['price_median'] = (
    train_exercise_expl
    .groupby(GR_COLS)['prices']
    .transform('median')
)

In [81]:
train_exercise_expl = (
    train_exercise_expl
    .assign(price_ratio = train_exercise_expl.prices / train_exercise_expl.price_median)
)

In [82]:
train_exercise_expl[:100]

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,POS,POS_price,is_clicked,price_mean,price_median,price_ratio
0,0004IOZI7CKF,0146f7cb014ba,1541266796,3,clickout item,2627602,DE,"Valencia, Spain",tablet,nan,88760,48,23,0,0,95.84,85.0,0.564706
1,0004IOZI7CKF,0146f7cb014ba,1541266796,3,clickout item,2627602,DE,"Valencia, Spain",tablet,nan,927635,55,8,1,0,95.84,85.0,0.647059
2,0004IOZI7CKF,0146f7cb014ba,1541266796,3,clickout item,2627602,DE,"Valencia, Spain",tablet,nan,85284,56,13,2,0,95.84,85.0,0.658824
3,0004IOZI7CKF,0146f7cb014ba,1541266796,3,clickout item,2627602,DE,"Valencia, Spain",tablet,nan,110976,70,0,3,0,95.84,85.0,0.823529
4,0004IOZI7CKF,0146f7cb014ba,1541266796,3,clickout item,2627602,DE,"Valencia, Spain",tablet,nan,3381482,72,9,4,0,95.84,85.0,0.847059
5,0004IOZI7CKF,0146f7cb014ba,1541266796,3,clickout item,2627602,DE,"Valencia, Spain",tablet,nan,2094980,74,5,5,0,95.84,85.0,0.870588
6,0004IOZI7CKF,0146f7cb014ba,1541266796,3,clickout item,2627602,DE,"Valencia, Spain",tablet,nan,88745,76,24,6,0,95.84,85.0,0.894118
7,0004IOZI7CKF,0146f7cb014ba,1541266796,3,clickout item,2627602,DE,"Valencia, Spain",tablet,nan,7822344,77,20,7,0,95.84,85.0,0.905882
8,0004IOZI7CKF,0146f7cb014ba,1541266796,3,clickout item,2627602,DE,"Valencia, Spain",tablet,nan,110979,78,6,8,0,95.84,85.0,0.917647
9,0004IOZI7CKF,0146f7cb014ba,1541266796,3,clickout item,2627602,DE,"Valencia, Spain",tablet,nan,5190190,78,18,9,0,95.84,85.0,0.917647


In [83]:
analysis = (
    train_exercise_expl
    .groupby(['POS','is_clicked'])
    .mean()
)

In [84]:
analysis

prices  POS_price  price_mean  price_median  price_ratio
POS is_clicked                                                              
0   0           137.288111  11.401516  123.010145    104.280287     1.336026
    1           118.944174  10.011035  120.099656    101.017040     1.239415
1   0           126.659752  11.365457  121.757731    102.828730     1.257385
    1           103.715909   8.612603  122.036758    103.483471     1.035048
2   0           130.620019  11.370722  121.989906    102.949836     1.284700
    1            95.520115   8.260057  119.817602    101.929598     0.985725
3   0           130.205288  11.547165  121.915280    102.859716     1.284268
    1            98.529740   8.202602  120.192734    102.405204     0.986734
4   0           127.353320  11.233283  121.633894    102.599839     1.281485
    1            97.804721   8.113734  124.971612    105.741416     0.968361
5   0           126.215172  11.343218  121.763020    102.722184     1.274787
    1            91.523002   8.021792  121.694929    102.175545     0.953732
6   0           127.307093  11.132806  121.949054    102.890226     1.236874
    1            86.333333   8.494152  115.250630     96.536550     0.961077
7   0           121.097274  11.192496  121.336776    102.343027     1.186886
    1           106.036496   8.405109  132.564883    112.381387     0.979143
8   0           124.510692  11.420469  121.875141    102.792910     1.209551
    1            98.967871   8.879518  118.230369    100.506024     1.070033
9   0           123.614739  11.350885  121.669508    102.728098     1.213672
    1           118.187755   8.873469  120.858849     99.648980     0.989680
10  0           122.257968  11.470092  121.521994    102.673614     1.213749
    1            97.951456   8.587379  119.695815    100.684466     0.954976
11  0           122.565797  11.585916  121.404109    102.716400     1.194087
    1            90.014085   7.638498  117.990518     97.624413     0.914553
12  0           119.774488  11.424849  121.170061    102.451558     1.170302
    1            90.827225   8.340314  112.574429     96.431937     0.900815
13  0           116.924815  11.389553  120.636693    102.014643     1.163434
    1           110.159509   8.607362  136.388661    119.886503     0.941890
14  0           120.281439  11.704079  120.882377    102.380039     1.172682
    1            95.384058   8.644928  124.510191    105.028986     0.950417
15  0           114.845674  11.461388  120.984298    102.455095     1.145392
    1            92.905063   8.949367  122.425487    106.262658     0.911570
16  0           116.912040  11.834444  120.910853    102.494880     1.156311
    1           109.921429   8.464286  137.401992    116.664286     0.901687
17  0           117.881936  11.753546  121.101664    102.657440     1.149766
    1            96.508772   8.236842  129.780257    114.473684     0.855297
18  0           116.237476  11.840126  121.089299    102.832934     1.141519
    1           130.250000   9.016667  124.995715    101.687500     1.402378
19  0           118.670144  12.034092  120.928849    102.714477     1.173419
    1           104.409091   8.988636  129.931507    109.022727     1.022758
20  0           118.306679  12.042501  120.985580    102.785493     1.167081
    1           106.415094   8.943396  142.421044    120.735849     0.916519
21  0           121.203101  12.142783  121.314858    103.120636     1.180035
    1           106.932039  10.466019  123.434289    105.266990     1.120730
22  0           116.817238  12.224412  121.101171    103.023124     1.145034
    1            96.620690   7.586207  132.952146    115.189655     0.871267
23  0           119.798036  12.352176  121.220348    103.184846     1.172498
    1           108.123596   9.831461  125.736723    105.443820     0.976703
24  0           115.461259  12.265950  121.288114    103.507190     1.138436
    1           131.089888   9.067416  132.118652    115.898876     1.057133

## Studying a formula to combine position and price effect

Apply formula than uses: `log(position)*price_ratio` in ascendent order

Using previous sample for now

In [85]:
train_exercise_expl = (
    train_exercise_expl
    .assign(position = train_exercise_expl.POS + 1)
)

In [86]:
train_exercise_expl = (
    train_exercise_expl
    .assign(log_position = np.log(train_exercise_expl.position))
)

In [87]:
train_exercise_expl = (
    train_exercise_expl
    .assign(formula_1 = train_exercise_expl.log_position * train_exercise_expl.price_ratio)
)

In [88]:
train_exercise_expl[:100]

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,...,prices,POS,POS_price,is_clicked,price_mean,price_median,price_ratio,position,log_position,formula_1
0,0004IOZI7CKF,0146f7cb014ba,1541266796,3,clickout item,2627602,DE,"Valencia, Spain",tablet,nan,...,48,23,0,0,95.84,85.0,0.564706,24,3.178054,1.794666
1,0004IOZI7CKF,0146f7cb014ba,1541266796,3,clickout item,2627602,DE,"Valencia, Spain",tablet,nan,...,55,8,1,0,95.84,85.0,0.647059,9,2.197225,1.421734
2,0004IOZI7CKF,0146f7cb014ba,1541266796,3,clickout item,2627602,DE,"Valencia, Spain",tablet,nan,...,56,13,2,0,95.84,85.0,0.658824,14,2.639057,1.738673
3,0004IOZI7CKF,0146f7cb014ba,1541266796,3,clickout item,2627602,DE,"Valencia, Spain",tablet,nan,...,70,0,3,0,95.84,85.0,0.823529,1,0.000000,0.000000
4,0004IOZI7CKF,0146f7cb014ba,1541266796,3,clickout item,2627602,DE,"Valencia, Spain",tablet,nan,...,72,9,4,0,95.84,85.0,0.847059,10,2.302585,1.950425
5,0004IOZI7CKF,0146f7cb014ba,1541266796,3,clickout item,2627602,DE,"Valencia, Spain",tablet,nan,...,74,5,5,0,95.84,85.0,0.870588,6,1.791759,1.559885
6,0004IOZI7CKF,0146f7cb014ba,1541266796,3,clickout item,2627602,DE,"Valencia, Spain",tablet,nan,...,76,24,6,0,95.84,85.0,0.894118,25,3.218876,2.878054
7,0004IOZI7CKF,0146f7cb014ba,1541266796,3,clickout item,2627602,DE,"Valencia, Spain",tablet,nan,...,77,20,7,0,95.84,85.0,0.905882,21,3.044522,2.757979
8,0004IOZI7CKF,0146f7cb014ba,1541266796,3,clickout item,2627602,DE,"Valencia, Spain",tablet,nan,...,78,6,8,0,95.84,85.0,0.917647,7,1.945910,1.785659
9,0004IOZI7CKF,0146f7cb014ba,1541266796,3,clickout item,2627602,DE,"Valencia, Spain",tablet,nan,...,78,18,9,0,95.84,85.0,0.917647,19,2.944439,2.701956


## Implementing formula algorithm

In [89]:
validation = pd.read_csv("~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/validation.csv")
validation_session_set = set(validation['session_id'])

In [90]:
test_data = test_data.loc[test_data['session_id'].isin(validation_session_set)]

In [91]:
def get_submission_target(df):
    """Identify target rows with missing click outs."""

    mask = df["reference"].isnull() & (df["action_type"] == "clickout item")
    df_out = df[mask]

    return df_out

In [92]:
def string_to_array(s):
    """Convert pipe separated string to array."""

    if isinstance(s, str):
        out = s.split("|")
    elif math.isnan(s):
        out = []
    else:
        raise ValueError("Value must be either string of nan")
    return out

In [93]:
def explode_item_price(df_in, col_explode1, col_explode2):
    """Explode column col_expl of array type into multiple rows."""

    df = df_in.copy()
    df.loc[:, col_explode1] = df[col_explode1].apply(string_to_array)
    df.loc[:, col_explode2] = df[col_explode2].apply(string_to_array)    

    df_out = pd.DataFrame(
        {col: np.repeat(df[col].values,
                        df[col_explode1].str.len())
         for col in df.columns.drop([col_explode1,col_explode2])}
    )

    df_out.loc[:, col_explode1] = np.concatenate(df[col_explode1].values)
    df_out.loc[:, col_explode2] = np.concatenate(df[col_explode2].values)  
    
    df_out.loc[:, col_explode1] = df_out[col_explode1].apply(int)
    df_out.loc[:, col_explode2] = df_out[col_explode2].apply(int)

    return df_out

In [94]:
def group_concat(df, gr_cols, col_concat):
    """Concatenate multiple rows into one."""

    df_out = (
        df
        .groupby(gr_cols)[col_concat]
        .apply(lambda x: ' '.join(x))
        .to_frame()
        .reset_index()
    )

    return df_out

In [95]:
df_target_price = get_submission_target(test_data)

In [96]:
df_explode_price = explode_item_price(df_target_price, 'impressions','prices')

In [97]:
df_explode_price

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,009RGHI3G9A3,f05ab0de907e2,1541570940,2,clickout item,NaN,IN,"Nathdwara, India",mobile,NaN,10884872,64
1,009RGHI3G9A3,f05ab0de907e2,1541570940,2,clickout item,NaN,IN,"Nathdwara, India",mobile,NaN,7065316,28
2,02AOAVF9PVYH,4a01c3afbc224,1541681278,46,clickout item,NaN,JP,"Yokohama, Japan",desktop,Hotel|Resort|Sort by Price,1451247,80
3,02AOAVF9PVYH,4a01c3afbc224,1541681278,46,clickout item,NaN,JP,"Yokohama, Japan",desktop,Hotel|Resort|Sort by Price,559056,81
4,02AOAVF9PVYH,4a01c3afbc224,1541681278,46,clickout item,NaN,JP,"Yokohama, Japan",desktop,Hotel|Resort|Sort by Price,1045096,81
5,02AOAVF9PVYH,4a01c3afbc224,1541681278,46,clickout item,NaN,JP,"Yokohama, Japan",desktop,Hotel|Resort|Sort by Price,1963879,82
6,02AOAVF9PVYH,4a01c3afbc224,1541681278,46,clickout item,NaN,JP,"Yokohama, Japan",desktop,Hotel|Resort|Sort by Price,693596,82
7,02AOAVF9PVYH,4a01c3afbc224,1541681278,46,clickout item,NaN,JP,"Yokohama, Japan",desktop,Hotel|Resort|Sort by Price,1967173,82
8,02AOAVF9PVYH,4a01c3afbc224,1541681278,46,clickout item,NaN,JP,"Yokohama, Japan",desktop,Hotel|Resort|Sort by Price,2403128,83
9,02AOAVF9PVYH,4a01c3afbc224,1541681278,46,clickout item,NaN,JP,"Yokohama, Japan",desktop,Hotel|Resort|Sort by Price,1052958,83


### Creating sorting variables

In [98]:
df_explode_price["POS"] = (
    df_explode_price
    .groupby(['user_id','session_id','timestamp','step'])
    .cumcount()
)

In [99]:
df_explode_price = (
    df_explode_price
    .assign(position = df_explode_price.POS + 1)
)

In [100]:
df_explode_price['price_mean'] = (
    df_explode_price
    .groupby(GR_COLS)['prices']
    .transform('mean')
)

In [101]:
df_explode_price = (
    df_explode_price
    .assign(log_position = np.log(df_explode_price.position))
)

In [102]:
df_explode_price = (
    df_explode_price
    .assign(price_ratio = df_explode_price.prices / df_explode_price.price_mean)
)

In [103]:
df_explode_price = (
    df_explode_price
    .assign(formula_1 = df_explode_price.log_position * df_explode_price.price_ratio)
)

### Sorting target data

In [104]:
# takes a lot of time
df_expl_price = (
    df_explode_price
    .groupby(['user_id','session_id','timestamp','step'])
    .apply(lambda x: x.sort_values(['formula_1','POS']))
    .reset_index(drop=True)
)
# takes a lot of time

In [105]:
df_expl_price

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,POS,position,price_mean,log_position,price_ratio,formula_1
0,0004Q49X39PY,9de47d9a66494,1541641157,1,clickout item,NaN,PH,"Iloilo City, Philippines",mobile,NaN,2213014,53,0,1,41.96,0.000000,1.263108,0.000000
1,0004Q49X39PY,9de47d9a66494,1541641157,1,clickout item,NaN,PH,"Iloilo City, Philippines",mobile,NaN,3184842,40,1,2,41.96,0.693147,0.953289,0.660769
2,0004Q49X39PY,9de47d9a66494,1541641157,1,clickout item,NaN,PH,"Iloilo City, Philippines",mobile,NaN,2292254,16,17,18,41.96,2.890372,0.381316,1.102144
3,0004Q49X39PY,9de47d9a66494,1541641157,1,clickout item,NaN,PH,"Iloilo City, Philippines",mobile,NaN,3812004,29,5,6,41.96,1.791759,0.691134,1.238347
4,0004Q49X39PY,9de47d9a66494,1541641157,1,clickout item,NaN,PH,"Iloilo City, Philippines",mobile,NaN,8153310,20,21,22,41.96,3.091042,0.476644,1.473328
5,0004Q49X39PY,9de47d9a66494,1541641157,1,clickout item,NaN,PH,"Iloilo City, Philippines",mobile,NaN,3202894,22,16,17,41.96,2.833213,0.524309,1.485479
6,0004Q49X39PY,9de47d9a66494,1541641157,1,clickout item,NaN,PH,"Iloilo City, Philippines",mobile,NaN,3505150,26,10,11,41.96,2.397895,0.619638,1.485826
7,0004Q49X39PY,9de47d9a66494,1541641157,1,clickout item,NaN,PH,"Iloilo City, Philippines",mobile,NaN,2227026,21,20,21,41.96,3.044522,0.500477,1.523712
8,0004Q49X39PY,9de47d9a66494,1541641157,1,clickout item,NaN,PH,"Iloilo City, Philippines",mobile,NaN,2712428,20,24,25,41.96,3.218876,0.476644,1.534259
9,0004Q49X39PY,9de47d9a66494,1541641157,1,clickout item,NaN,PH,"Iloilo City, Philippines",mobile,NaN,1984229,28,12,13,41.96,2.564949,0.667302,1.711596


In [106]:
df_expl_price = df_expl_price[GR_COLS + ["impressions"]]


In [107]:
df_out1 = df_expl_price.assign(impressions=lambda x: x["impressions"].apply(str))

In [108]:
df_out = group_concat(df_out1, GR_COLS, 'impressions')


In [109]:
df_out.rename(columns={'impressions': 'item_recommendations'}, inplace=True)
df_out[:10]

,user_id,session_id,timestamp,step,item_recommendations
0,0004Q49X39PY,9de47d9a66494,1541641157,1,2213014 3184842 2292254 3812004 8153310 320289...
1,0004Q49X39PY,beea5c27030cb,1541561202,1,3812004 3505150 2292254 3202894 4476010 710135...
2,00071784XQ6B,9617600e1ba7c,1541630328,2,22721 22854 16121 22819 3067559 22764 1478189 ...
3,0008BO33KUQ0,2d0e2102ee0dc,1541636411,6,507861 2176280 1669587 4272108 3133204 502066 ...
4,000GO9NY6P4M,55dbafdbb9bab,1541594662,2,157710 1287958 160577 319866 483691 1618677 40...
5,000IRHJS2DL9,f6ffffd20d43d,1541605541,12,102540 33191 2285010 20144 2547840 4481844 201...
6,000JB0UNEH23,7df07dc9fe26e,1541618174,1,2184212 2756867 4063406 9503248 3884098 290999...
7,000OWRCYEHKT,53e84da5c2dad,1541706095,3,5416250 7795438 3926654 8989706 6797450 722521...
8,000VBY1D6BP8,033fddaaa99af,1541587306,4,32256 5411980 32246 6865398 32230 5592656 3223...
9,00130PO6XSZR,0d5fe519827f3,1541701777,1,9416426 5828214 2909952 3050704 5209224 386938...


In [ ]:
df_out.to_csv("~/Documents/Tesi/trivagoRecSysChallengeData2019_v2/formula1.csv", index=False)